In [5]:
# !pip install fastf1

In [1]:
import logging

logging.getLogger('fastf1').setLevel(logging.ERROR)

In [17]:
import fastf1
import pandas as pd
import numpy as np
from pathlib import Path
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

fastf1.Cache.enable_cache('cache')

# Loading data

Data - attributes from session <br>
[attr for attr in dir(session) if not attr.startswith('_')]

['api_path', <br>
 'car_data',<br>
 'date',<br>
 'drivers',<br>
 'event',<br>
 'f1_api_support',<br>
 'get_circuit_info',<br>
 'get_driver',<br>
 'laps',<br>
 'load',<br>
 'name',<br>
 'pos_data',<br>
 'race_control_messages',<br>
 'results',<br>
 'session_info',<br>
 'session_start_time',<br>
 'session_status',<br>
 't0_date',<br>
 'total_laps',<br>
 'track_status',<br>
 'weather_data']

In [6]:
def append_df(df, path):
    df.to_csv(path, mode='a', index=False, header=not path.exists())

In [9]:
OUT = Path(r"C:\Users\Ogi\Desktop\Vezbanje\FON\Master rad\Aplikacija\data\Model2")
OUT.mkdir(exist_ok=True)

In [11]:
files = {
    'laps': OUT / 'laps.csv',
    'weather': OUT / 'weather.csv',
    'track': OUT / 'track_status.csv',
    'drivers': OUT / 'drivers.csv',
    'events': OUT / 'events.csv'
}

In [29]:
for season in range(2020, 2025):
    calendar = fastf1.get_event_schedule(season)

    for _, row in calendar.iterrows():
        try:
            session = fastf1.get_session(season, row['EventName'], 'R')
    
            session.load(
                telemetry=False,
                messages=False
            )
    
            # --- EVENT / TRACK INFO ---
            event = session.event.copy()
            event['Season'] = season
            append_df(event, files['events'])
    
            # --- DRIVERS ---
            for drv in session.drivers:
                d = session.get_driver(drv)
                d['Season'] = season
                d['Round'] = row['RoundNumber']
                append_df(pd.DataFrame([d]), files["drivers"])
            
            # --- LAPS ---
            laps = session.laps.copy()
            laps['Season'] = season
            laps['Round'] = row['RoundNumber']
            laps['EventName'] = row['EventName']
            append_df(laps, files['laps'])
            
            # --- WEATHER ---
            if session.weather_data is not None:
                weather = session.weather_data.copy()
                weather['Season'] = season
                weather['Round'] = row['RoundNumber']
                append_df(weather, files['weather'])
    
            # --- TRACK STATUS ---
            if session.track_status is not None:
                ts = session.track_status.copy()
                ts['Season'] = season
                ts['Round'] = row['RoundNumber']
                append_df(ts, files['track'])

            print(f"✅ {season} {row['EventName']}")
            
        except Exception as e:
            print(f"***Skipped {season} {row['EventName']}: {e}")

Skipped races: azerbajian, singapore, us, maxico, brazil, las vegas, qatar, abu dhabi in 2024.


In [30]:
df_drivers = pd.read_csv('data/Model2/drivers.csv').convert_dtypes()
df_laps = pd.read_csv('data/Model2/laps.csv').convert_dtypes()
df_track_status = pd.read_csv('data/Model2/track_status.csv').convert_dtypes()
df_weather = pd.read_csv('data/Model2/weather.csv').convert_dtypes()

In [26]:
df_drivers.dtypes

DriverNumber                   Int64
BroadcastName         string[python]
Abbreviation          string[python]
DriverId              string[python]
TeamName              string[python]
TeamColor             string[python]
TeamId                string[python]
FirstName             string[python]
LastName              string[python]
FullName              string[python]
HeadshotUrl           string[python]
CountryCode           string[python]
Position                       Int64
ClassifiedPosition    string[python]
GridPosition                   Int64
Q1                             Int64
Q2                             Int64
Q3                             Int64
Time                  string[python]
Status                string[python]
Points                       Float64
Laps                           Int64
Season                         Int64
Round                          Int64
dtype: object

In [25]:
df_laps.dtypes

Time                  string[python]
Driver                string[python]
DriverNumber                   Int64
LapTime               string[python]
LapNumber                      Int64
Stint                          Int64
PitOutTime            string[python]
PitInTime             string[python]
Sector1Time           string[python]
Sector2Time           string[python]
Sector3Time           string[python]
Sector1SessionTime    string[python]
Sector2SessionTime    string[python]
Sector3SessionTime    string[python]
SpeedI1                        Int64
SpeedI2                        Int64
SpeedFL                        Int64
SpeedST                        Int64
IsPersonalBest               boolean
Compound              string[python]
TyreLife                       Int64
FreshTyre                    boolean
Team                  string[python]
LapStartTime          string[python]
LapStartDate                   Int64
TrackStatus                    Int64
Position                       Int64
D

In [27]:
df_track_status.dtypes

Time       string[python]
Status              Int64
Message    string[python]
Season              Int64
Round               Int64
dtype: object

In [28]:
df_weather.dtypes

Time             string[python]
AirTemp                 Float64
Humidity                Float64
Pressure                Float64
Rainfall                boolean
TrackTemp               Float64
WindDirection             Int64
WindSpeed               Float64
Season                    Int64
Round                     Int64
dtype: object